In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import keras_tuner as kt

In [4]:
df = pd.read_csv('Unit_1.csv')
df = df.drop(['Unnamed: 0'], axis=1)

#### Multivariate Regression using DNN

In [11]:
X = df.drop('U1_ActivePower', axis=1)
y = df['U1_ActivePower']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=123)

In [13]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [14]:
model = Sequential()
model.add(Dense(128, input_dim=4, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

In [15]:
model.compile(loss='mse', optimizer=Adam(0.0001), metrics=['mae'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 128)               640       
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 13,121
Trainable params: 13,121
Non-trainable params: 0
_________________________________________________________________


In [16]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=300, batch_size=128)

Epoch 1/300


c:\Users\deguz\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\data_adapter.py:1699: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


159/159 [==============================] - 2s 4ms/step - loss: 6908.3389 - mae: 83.0918 - val_loss: 6599.5103 - val_mae: 81.2138
Epoch 2/300
159/159 [==============================] - 0s 3ms/step - loss: 5582.1792 - mae: 74.4184 - val_loss: 4104.4438 - val_mae: 63.6110
Epoch 3/300
159/159 [==============================] - 0s 3ms/step - loss: 2379.8743 - mae: 45.9746 - val_loss: 1058.4993 - val_mae: 29.0267
Epoch 4/300
159/159 [==============================] - 0s 3ms/step - loss: 715.4733 - mae: 22.1496 - val_loss: 569.1254 - val_mae: 19.5535
Epoch 5/300
159/159 [==============================] - 0s 2ms/step - loss: 516.5609 - mae: 18.7126 - val_loss: 470.2533 - val_mae: 17.9432
Epoch 6/300
159/159 [==============================] - 0s 2ms/step - loss: 434.4402 - mae: 17.2149 - val_loss: 396.2138 - val_mae: 16.4381
Epoch 7/300
159/159 [==============================] - 0s 2ms/step - loss: 367.8452 - mae: 15.8262 - val_loss: 333.5494 - val_mae: 15.0736
Epoch 8/300
159/159 [============

#### Hyperparameter tuning using Keras Tuner

In [41]:
def model_builder(hp):
  model = tf.keras.Sequential()
  hp_units1 = hp.Int('units1', min_value=32, max_value=1024, step=32)
  hp_units2 = hp.Int('units2', min_value=32, max_value=1024, step=32)
  hp_units3 = hp.Int('units3', min_value=32, max_value=1024, step=32)
  model.add(tf.keras.layers.Dense(units=hp_units1, input_dim=4, activation='relu'))
  model.add(tf.keras.layers.Dense(units=hp_units2, activation='relu'))
  model.add(tf.keras.layers.Dense(units=hp_units3, activation='relu'))
  model.add(tf.keras.layers.Dense(1))
  
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                loss='mse',
                metrics=['mae'])

  return model

In [42]:
tuner = kt.Hyperband(model_builder,
                     objective='val_mae',
                     max_epochs=10,
                     directory='tuner_dir',
                     project_name='magat_kt')

INFO:tensorflow:Reloading Oracle from existing project tuner_dir\magat_kt\oracle.json
INFO:tensorflow:Reloading Tuner from tuner_dir\magat_kt\tuner0.json


In [43]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_mae', patience=7)

In [44]:
tuner.search(X_train, y_train, epochs=10, validation_split=0.2, callbacks=[stop_early])

Trial 90 Complete [00h 00m 14s]
val_mae: 0.430368036031723

Best val_mae So Far: 0.34634190797805786
Total elapsed time: 00h 17m 36s
INFO:tensorflow:Oracle triggered exit


In [45]:
for h_param in [f"units{i}" for i in range(1,4)] + ['learning_rate']:
  print(h_param, tuner.get_best_hyperparameters()[0].get(h_param))

units1 896
units2 256
units3 224
learning_rate 0.001


#### Retraining

In [55]:
model = Sequential()
model.add(Dense(896, input_dim=4, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(224, activation='relu'))
model.add(Dense(1))

In [56]:
model.compile(loss='mse', optimizer=Adam(0.001), metrics=['mae'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 896)               4480      
                                                                 
 dense_13 (Dense)            (None, 256)               229632    
                                                                 
 dense_14 (Dense)            (None, 224)               57568     
                                                                 
 dense_15 (Dense)            (None, 1)                 225       
                                                                 
Total params: 291,905
Trainable params: 291,905
Non-trainable params: 0
_________________________________________________________________


In [57]:
checkpointer = ModelCheckpoint(filepath='checkpoint_dir/training_0.ckpt', 
                               monitor='val_mae', mode='min',
                               verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_mae', mode='min', verbose=1, patience=20)
reduce_lr = ReduceLROnPlateau(monitor='val_mae', factor=0.1, patience=7, min_lr=0.00001, verbose=1)
callbacks=[early_stopping, reduce_lr, checkpointer]

In [58]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=1000, callbacks=callbacks)

Epoch 1/1000


c:\Users\deguz\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\data_adapter.py:1699: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


625/636 [============================>.] - ETA: 0s - loss: 271.1884 - mae: 6.7633
Epoch 1: val_mae improved from inf to 0.59608, saving model to checkpoint_dir\training_0.ckpt
INFO:tensorflow:Assets written to: checkpoint_dir\training_0.ckpt\assets
636/636 [==============================] - 2s 3ms/step - loss: 266.5719 - mae: 6.6609 - val_loss: 0.9418 - val_mae: 0.5961 - lr: 0.0010
Epoch 2/1000
621/636 [============================>.] - ETA: 0s - loss: 0.7893 - mae: 0.5208
Epoch 2: val_mae improved from 0.59608 to 0.40949, saving model to checkpoint_dir\training_0.ckpt
INFO:tensorflow:Assets written to: checkpoint_dir\training_0.ckpt\assets
636/636 [==============================] - 2s 3ms/step - loss: 0.7891 - mae: 0.5193 - val_loss: 0.5930 - val_mae: 0.4095 - lr: 0.0010
Epoch 3/1000
624/636 [============================>.] - ETA: 0s - loss: 0.7376 - mae: 0.5076
Epoch 3: val_mae did not improve from 0.40949
636/636 [==============================] - 1s 2ms/step - loss: 0.7440 - mae: 0

#### Going Deeper

In [83]:
model = Sequential()
model.add(Dense(1024, input_dim=4, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1))

In [84]:
model.compile(loss='mse', optimizer=Adam(0.01), metrics=['mae'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_46 (Dense)            (None, 1024)              5120      
                                                                 
 dense_47 (Dense)            (None, 512)               524800    
                                                                 
 dense_48 (Dense)            (None, 256)               131328    
                                                                 
 dense_49 (Dense)            (None, 128)               32896     
                                                                 
 dense_50 (Dense)            (None, 1)                 129       
                                                                 
Total params: 694,273
Trainable params: 694,273
Non-trainable params: 0
_________________________________________________________________


In [85]:
checkpointer = ModelCheckpoint(filepath='checkpoint_dir/training_1.ckpt', 
                               monitor='val_mae', mode='min',
                               verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
reduce_lr = ReduceLROnPlateau(monitor='val_mae', factor=0.1, patience=3, min_lr=0.00001, verbose=1)
callbacks=[early_stopping, checkpointer, reduce_lr]

In [86]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=1000, callbacks=callbacks)

Epoch 1/1000


c:\Users\deguz\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\data_adapter.py:1699: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


620/636 [============================>.] - ETA: 0s - loss: 319.9737 - mae: 5.4202
Epoch 1: val_mae improved from inf to 0.95133, saving model to checkpoint_dir\training_1.ckpt
INFO:tensorflow:Assets written to: checkpoint_dir\training_1.ckpt\assets
636/636 [==============================] - 2s 3ms/step - loss: 312.0320 - mae: 5.3097 - val_loss: 1.4920 - val_mae: 0.9513 - lr: 0.0100
Epoch 2/1000
614/636 [===========================>..] - ETA: 0s - loss: 1.6679 - mae: 0.9464
Epoch 2: val_mae did not improve from 0.95133
636/636 [==============================] - 2s 2ms/step - loss: 1.7404 - mae: 0.9659 - val_loss: 2.5188 - val_mae: 1.2392 - lr: 0.0100
Epoch 3/1000
620/636 [============================>.] - ETA: 0s - loss: 2.4898 - mae: 1.1819
Epoch 3: val_mae did not improve from 0.95133
636/636 [==============================] - 2s 2ms/step - loss: 2.4757 - mae: 1.1781 - val_loss: 2.4256 - val_mae: 1.1278 - lr: 0.0100
Epoch 4/1000
623/636 [============================>.] - ETA: 0s - los

#### Loading the saved weights

In [90]:
model = Sequential()
model.add(Dense(1024, input_dim=4, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1))

In [91]:
model.compile(loss='mse', optimizer=Adam(0.01), metrics=['mae'])

In [92]:
model.load_weights('checkpoint_dir/training_1.ckpt')

In [94]:
score = model.evaluate(X_test, y_test)

341/341 [==============================] - 1s 2ms/step - loss: 0.5640 - mae: 0.3285


#### Saving the Model

In [122]:
model.save('saved_model/dnn_unit1')

INFO:tensorflow:Assets written to: saved_model/dnn_unit1\assets


In [123]:
new_model = tf.keras.models.load_model('saved_model/dnn_unit1')


In [124]:
new_model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_80 (Dense)            (None, 128)               768       
                                                                 
 dense_81 (Dense)            (None, 256)               33024     
                                                                 
 dense_82 (Dense)            (None, 512)               131584    
                                                                 
 dense_83 (Dense)            (None, 1024)              525312    
                                                                 
 dense_84 (Dense)            (None, 1)                 1025      
                                                                 
Total params: 691,713
Trainable params: 691,713
Non-trainable params: 0
_________________________________________________________________


In [125]:
new_model.evaluate(X_test, y_test)

341/341 [==============================] - 1s 2ms/step - loss: 0.5603 - mae: 0.3266


[0.560325562953949, 0.3265897035598755]